In [1]:
! pip install --upgrade ir_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 5.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.3/294.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unlzw3: filename=unlzw3-0.2.1-py3-none-any.whl size=6066 sha256=4769cf6e1fed0492549cbae348b2970e465cd52f03e8fd4c477b36cfedc4da72
  Stored in directory: /root/.cache/pip/wheels/e1/3c/d9/e33962c8aad8999dc5560b6e71baafa2335c269f532f5e176a
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18922 sha256=56e8f

In [2]:
#! python -m spacy download fr_core_news_sm
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 50.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.4 requires h5py~=3.1.0, but you have h5py 3.7.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires tensorboard<2.7,>=2.6.0, but you have tensorboard 2.10.1 which is incompatible.
tensorflow 2.6.4 requires typing-extensions<3.11,>=3.7, but you have typing-extensions 4.1.1 which is incompatible.
tensorflow-transform 1.9.0 requires pyarrow<6,>=1

**process class**

In [2]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy
import re
import numpy as np
import math

**PreProcessing**

In [3]:
def RM_Punc(text):
    punctuation_pattern = re.compile('[^A-z ]')
    no_punct_text = [re.sub(punctuation_pattern, '', token) for token in text]
    return ''.join([token for token in no_punct_text if token != ''])

def preSomthing(text):
    text = text.lower()
    text = RM_Punc(text)
    return text
    
def RM_Stop_Word(tokens):
    # check in lowercase 
    t = [token for token in tokens if token.lower() not in stopwords.words("english")]
      
    return t

nlp = spacy.load('en_core_web_sm')
def Lemm(text):
    
    doc = nlp(text)
    return [i.lemma_ for i in doc]

def PreProcessing(text):
    text = preSomthing(text)
    tokens= Lemm(text)
    return RM_Stop_Word(tokens)



In [9]:
text = "hello my friend ,my name is abdou benomar ,my friend sent me to search for his glasses "
text =preSomthing(text)
token = Lemm(text)
token

['hello',
 'my',
 'friend',
 'my',
 'name',
 'be',
 'abdou',
 'benomar',
 'my',
 'friend',
 'send',
 'I',
 'to',
 'search',
 'for',
 'his',
 'glass']

In [11]:
token= RM_Stop_Word(tokens=token)
token

['hello',
 'friend',
 'name',
 'abdou',
 'benomar',
 'friend',
 'send',
 'search',
 'glass']

**indexation**

In [ ]:

vector=[]
def indexVect(token):
    tokens= token.copy()
    
    index=[]
    som = len(tokens)
    for i in token :
        
        indexVal = tokens.count(i)
        
        
        [tokens.pop(tokens.index(item)) for item in tokens if item ==i]  
        
        if i not in vector:
            vector.append(i)
                
        index.append([vector.index(i),indexVal/som])
    return index   

**get score docs**

In [7]:
import ir_datasets
datasetDoc = ir_datasets.load("wikir/en1k")
for doc in datasetDoc.docs_iter():
    doc # namedtuple<doc_id, url, title, abstract>

[INFO] If you have a local copy of https://zenodo.org/record/3565761/files/wikIR1k.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/554299bca984640cb283d6ba55753608
[INFO] [starting] https://zenodo.org/record/3565761/files/wikIR1k.zip
[INFO] [finished] https://zenodo.org/record/3565761/files/wikIR1k.zip: [00:06] [165MB] [23.6MB/s]


In [8]:
datasetQuery = ir_datasets.load("wikir/en1k/test")
for query in datasetQuery.queries_iter():
    query # namedtuple<query_id, text>

In [ ]:
docs = []
docsID = []

for i in datasetDoc.docs_iter()[:60000]:
   
    docs.append(indexVect(PreProcessing(i.text)))
    docsID.append(i.doc_id)

1781133
2426736
2224122
219642
1728654
1889917
1518473
1459418
1102956
931408
1043905
488327
1069841
1601336
1415191
1531137
912052
935016
1671729
1946935
752649
1634473
867034
1064413
1612321
2222141
1784663
2285161
1445565
1469713
1466972
963723
897432
1218994
839556
2170795
1024270
1336432
2087053
421312
547088
636479
2166683
833090
2363592
2289234
1370298
1481866
1763284
1876607
729433
352078
2354924
2416742
1283366
1602376
1301443
1962423
2436983
79923
490168
502902
1738343
52687
503290
443815
2259227
1375558
488452
341885
2329563
328166
2305145
2243061
705605
1548811
1316555
1924943
350851
526876
759572
747365
1416073
2161217
1760290
1915270
1051703
1052256
51110
890294
1500765
1020455
495371
452464
1222862
1008941
1313953
414617
578027
1834363
247020
42650
404210
670777
1922332
1682663
827043
2124360
608864
867166
1146005
1716457
1580743
22915
581042
396072
1019762
275841
718113
1790396
2384572
2431285
73954
1749729
114691
1104339
268946
153113
134078
464565
2360440
495715
91198

In [ ]:
len(docsID)

In [11]:
Query=[]
QueryID=[]

for i in datasetQuery.queries_iter():
        Query.append(indexVect(PreProcessing(i.text)))
        QueryID.append(i.query_id)

In [12]:
len(Query)


100

**using cosinus**

In [14]:

def tfCos(dnp, qnp):   
    inner=0.0
    p1=0.0
    p2=0.0
    if len(qnp) == 0 or len(dnp)==0:
        return 0
    for i in qnp[:,:1]:
        
        if i in dnp[:,:1]:
            
            qi= np.where(qnp[:,:1]==i)
            
            di= np.where(dnp[:,:1]==i)
          
            inner = inner + (dnp[di[0]][:,1:2][0][0] * qnp[qi[0]][:,1:2][0][0]  )
            p1= p1 + dnp[di[0]][:,1:2][0][0]**2
            p2= p2 + qnp[qi[0]][:,1:2][0][0]**2
            
    if p1==0.0 or p2 ==0.0:
        return 0  
    cos = (inner) /( math.sqrt(p1) * math.sqrt(p2) )
    return cos    


test cos

In [15]:
Rcos ={}

for i in range(len(Query)):
    R=[]
    for j in range(len(docs)):
    
        iq = np.array( Query[i])
    
        DAi = np.array(docs[j] )
    
        R.append( [ docsID[j] , tfCos( DAi,iq)])
    Rcos[QueryID[i]] =R



In [34]:
len(Rcos['158491'])

5000

In [18]:
scores = []
for score in datasetQuery.scoreddocs_iter():
    
    scores.append(score)
        
len(scores)

10000

In [19]:
sc=[]
for i in scores:
    if i.doc_id in docsID:
        sc.append(i)

In [32]:
treash =Rcos[list(Rcos.keys())[0]]
n = np.array(treash)
len(n)

5000

In [30]:
nbDocsPer=0
nbDocReturne=0
nbDocRetPer=0
for i in list(Rcos.keys()):
    trash = Rcos[i]
    n = np.array(trash)
    for j in sc:
        ind = np.where(n[:,:1] == j.doc_id)
        row = n[ind[0]][0][1]
        score= j.score
        if int(row[0])>0 and score >0:
            nbDocRetPer=nbDocRetPer+1
            
        if int(row[0])>0:
            nbDocReturne=nbDocReturne+1
            
        if score>0:
            nbDocsPer=nbDocsPer+1
       
        
        
       
print(nbDocsPer,nbDocReturne,nbDocRetPer)

10100 539 539


In [179]:
def tfInner(dnp , qnp):   
    if len(qnp) == 0 or len(dnp)==0:
        return 0
    
    inner=0.0

    for i in qnp[:,:1]:
        if i in dnp[:,:1]:
            qi= np.where(qnp[:,:1]==i)
            di= np.where(dnp[:,:1]==i)
            inner = inner + (dnp[di[0]][:,1:2][0][0] * qnp[qi[0]][:,1:2][0][0]  )
    return inner


**dice**

In [ ]:
def tfDice(dnp,qnp):   
        
    inner=0.0
    p1=0.0
    p2=0.0
    for i in qnp[:,:1]:
        if i in dnp[:,:1]:
            qi= np.where(qnp[:,:1]==i)
            di= np.where(dnp[:,:1]==i)
            inner = inner + (dnp[di[0]][:,1:2][0][0] * qnp[qi[0]][:,1:2][0][0]  )
            p1= p1 + dnp[di[0]][:,1:2][0][0]**2
            p2=p2 + qnp[qi[0]][:,1:2][0][0]**2
    if (p1+p2 )   ==0.0 :
        return 0        
    dice = (2*inner) / (p1+p2 )        
    return dice    


In [186]:
sm=0
for i in range(len(Rcos)):
    if Rcos[i]>0 and Rinner[i] >0: 
        sm=sm+1
    if Rcos[i]== 0 and Rinner[i] ==0 :
        sm=sm+1

        
print(sm)
print(len(Rcos))

144
144
